In [2]:
!pip install --upgrade kaggle

from pathlib import Path
import json, os, re, subprocess, zipfile, tarfile
from urllib.parse import urlparse
import requests

def download_datasets_from_json(
    json_path,
    root="/home/datasets",
    force=False,
    unzip=True,
    timeout=60,
    kaggle_username=None,   # <-- NEW
    kaggle_key=None,        # <-- NEW (token)
    persist_credentials=False,  # <-- NEW: write ~/.kaggle/kaggle.json if True
):
    """
    Read datasets.json and download into `root` (one folder per dataset).
    Accepts Kaggle credentials directly via arguments.

    - Kaggle URLs use Kaggle CLI (datasets/competitions)
    - Direct .zip/.tar* URLs are downloaded via HTTP
    - Others get a README.txt with the source links
    """

    # ---------- helpers ----------
    def slugify(text: str) -> str:
        text = text.lower()
        text = re.sub(r"&", " and ", text)
        text = re.sub(r"[^a-z0-9]+", "-", text).strip("-")
        return re.sub(r"-{2,}", "-", text)

    def set_kaggle_credentials(user: str, key: str, persist: bool):
        # Always set env vars for this process
        os.environ["KAGGLE_USERNAME"] = user
        os.environ["KAGGLE_KEY"] = key

        if persist:
            kd = Path.home() / ".kaggle"
            kd.mkdir(parents=True, exist_ok=True)
            kf = kd / "kaggle.json"
            kf.write_text(json.dumps({"username": user, "key": key}), encoding="utf-8")
            try:
                kf.chmod(0o600)
            except Exception:
                pass  # best-effort on non-POSIX
            # Don't print creds

    def have_kaggle_cli() -> bool:
        try:
            cp = subprocess.run(["kaggle", "--version"], capture_output=True, text=True)
            if cp.returncode != 0:
                return False
        except FileNotFoundError:
            return False
        # Accept either file or env vars
        creds_file = Path.home() / ".kaggle" / "kaggle.json"
        has_env = bool(os.environ.get("KAGGLE_USERNAME") and os.environ.get("KAGGLE_KEY"))
        return creds_file.exists() or has_env

    def kaggle_info_from_url(url: str):
        p = urlparse(url)
        if "kaggle.com" not in (p.netloc or ""):
            return None, None
        parts = [x for x in (p.path or "").split("/") if x]
        if not parts:
            return None, None
        if parts[0] == "datasets" and len(parts) >= 3:
            return "dataset", f"{parts[1]}/{parts[2]}"
        if parts[0] == "competitions" and len(parts) >= 2:
            return "competition", parts[1]
        return None, None

    def ensure_dir(p: Path): p.mkdir(parents=True, exist_ok=True)
    def is_nonempty_dir(p: Path) -> bool: return p.exists() and any(p.iterdir())

    def write_readme(folder: Path, name: str, urls: list[str], extra: str = ""):
        ensure_dir(folder)
        readme = folder / "README.txt"
        lines = [name, "", "Links:"]
        for u in urls: lines.append(f"- {u}")
        if extra: lines += ["", extra]
        readme.write_text("\n".join(lines) + "\n", encoding="utf-8")
        print(f"📝 Wrote {readme}")

    def unzip_all_in_dir(d: Path):
        for z in sorted(d.glob("*.zip")):
            print(f"📦 Unzipping {z.name}")
            with zipfile.ZipFile(z, "r") as zf:
                zf.extractall(d)
            z.unlink()
        for t in sorted(list(d.glob("*.tar")) +
                        list(d.glob("*.tar.gz")) + list(d.glob("*.tgz")) +
                        list(d.glob("*.tar.bz2")) + list(d.glob("*.tbz2")) +
                        list(d.glob("*.tar.xz"))):
            print(f"📦 Extracting {t.name}")
            with tarfile.open(t, "r:*") as tf:
                tf.extractall(d)
            t.unlink()

    def http_download(url: str, dest_dir: Path) -> Path | None:
        ensure_dir(dest_dir)
        name = os.path.basename(urlparse(url).path) or "download.bin"
        out = dest_dir / name
        print(f"⬇️  HTTP: {url}\n    → {out}")
        try:
            with requests.get(url, stream=True, timeout=timeout) as r:
                r.raise_for_status()
                with open(out, "wb") as f:
                    for chunk in r.iter_content(chunk_size=1024 * 1024):
                        if chunk: f.write(chunk)
            return out
        except Exception as e:
            print(f"❌ HTTP download failed: {e}")
            return None

    def maybe_direct_download(url: str) -> bool:
        lower = url.lower()
        looks_file = any(lower.endswith(ext) for ext in
                         (".zip", ".tar", ".tar.gz", ".tgz", ".tar.bz2", ".tbz2", ".tar.xz"))
        if not looks_file:
            return False
        fpath = http_download(url, target)
        if not fpath:
            return False
        if unzip:
            unzip_all_in_dir(target)
        return True

    def kaggle_download(mode: str, slug: str, dest: Path) -> bool:
        ensure_dir(dest)
        if mode == "dataset":
            cmd = ["kaggle", "datasets", "download", "-d", slug, "-p", str(dest), "-o"]
        else:
            cmd = ["kaggle", "competitions", "download", "-c", slug, "-p", str(dest), "-o"]
        print("→", " ".join(cmd))
        cp = subprocess.run(cmd, capture_output=True, text=True)
        print(cp.stdout or "")
        if cp.returncode != 0:
            if cp.stderr: print(cp.stderr)
            print("❌ Kaggle download failed (check credentials / accept competition rules).")
            return False
        if unzip:
            unzip_all_in_dir(dest)
        return True

    # ---------- apply creds if provided ----------
    if kaggle_username and kaggle_key:
        set_kaggle_credentials(kaggle_username, kaggle_key, persist_credentials)

    # ---------- load JSON ----------
    json_path = Path(json_path).expanduser().resolve()
    if not json_path.is_file():
        raise FileNotFoundError(f"datasets.json not found at: {json_path}")
    items = json.loads(json_path.read_text(encoding="utf-8")).get("datasets")
    if not isinstance(items, list):
        raise ValueError("JSON must contain top-level key 'datasets' as a list.")

    # ---------- root ----------
    root = Path(root).expanduser().resolve()
    ensure_dir(root)

    kaggle_ready = have_kaggle_cli()
    if not kaggle_ready:
        print("ℹ️ Kaggle CLI not available or no creds — Kaggle downloads will be skipped.")

    created = []
    for d in sorted(items, key=lambda x: x.get("id", 0)):
        did  = d.get("id")
        name = d.get("name", f"dataset_{did or 'x'}")
        urls = d.get("urls", [])
        folder = root / f"dataset{did}_{slugify(name)}"
        ensure_dir(folder)

        if is_nonempty_dir(folder) and not force:
            print(f"✅ Exists (skipping): {folder}")
            created.append(folder); continue

        print(f"\n=== [{did}] {name} → {folder} ===")
        target = folder

        success = False
        # Kaggle (if CLI + creds present)
        if kaggle_ready:
            for u in urls:
                mode, slug = kaggle_info_from_url(u)
                if mode and slug:
                    success = kaggle_download(mode, slug, target)
                    if success: break

        # Direct archives
        if not success:
            for u in urls:
                if maybe_direct_download(u):
                    success = True
                    break

        if not success:
            extra = ("Manual step required. Follow the links above, download files, "
                     "and place them in this folder (then unzip here).")
            write_readme(target, name, urls, extra=extra)

        created.append(folder)

    print("\n🎉 Done.")
    return created



Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
import sys, os, shutil, pathlib, subprocess

# install kaggle into the current kernel's env
!{sys.executable} -m pip install --upgrade kaggle

# make sure ~/.local/bin is on PATH (that’s where the kaggle script often lands)
local_bin = os.path.expanduser("~/.local/bin")
if local_bin not in os.environ.get("PATH",""):
    os.environ["PATH"] += os.pathsep + local_bin

print("kaggle binary:", shutil.which("kaggle"))
!kaggle --version

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
kaggle binary: /home/kristoffel/.local/bin/kaggle
Kaggle API 1.7.4.5


In [4]:
download_datasets_from_json(
    json_path="/home/kristoffel/datasets/datasets.json",
    root="/home/kristoffel/datasets",
    kaggle_username="krisslarsen",
    kaggle_key="YOUR_TOKEN_HERE",           # <- replace
    persist_credentials=True,               # writes ~/.kaggle/kaggle.json
    force=True,                             # <- re-download into existing folders
    unzip=True
)




=== [1] Nutrition5K → /home/kristoffel/datasets/dataset1_nutrition5k ===
📝 Wrote /home/kristoffel/datasets/dataset1_nutrition5k/README.txt

=== [2] Food101 → /home/kristoffel/datasets/dataset2_food101 ===
→ kaggle datasets download -d dansbecker/food-101 -p /home/kristoffel/datasets/dataset2_food101 -o
Dataset URL: https://www.kaggle.com/datasets/dansbecker/food-101
License(s): other


📦 Unzipping food-101.zip


EOFError: 